In [8]:
import pandas as pd
import numpy as np


def cleanData(file_path):
    # Reading the file

    # Load the data from the CSV file
    data = pd.read_csv(file_path)

    # Removing the specified columns and reordering the 'Item Number – 8 digit' column
    columns_to_remove = ['Item Number – 6 digit', 'UPC Number', 'Constant', 
                        'Customer-Specific Item Number', 'Pack Size Divisor', 
                        'RX/OTC Indicator']

    # Removing the columns
    data_cleaned = data.drop(columns=columns_to_remove)

    # Reordering 'Item Number – 8 digit' to the left
    column_to_move = data_cleaned.pop('Item Number – 8 digit')
    data_cleaned.insert(0, 'Item Number – 8 digit', column_to_move)

    # Moving all price columns and the contract flag to the right
    columns_to_move = ['AWP Price', 'Acquisition Price', 'Retail Price', 'WAC Price', 'Contract Flag']
    for col in columns_to_move:
        data_cleaned[col] = data_cleaned.pop(col)

    import re

    # Function to split the generic description into generic name and form
    def split_description(desc):
        match = re.search(r'[A-Z]', desc)
        if match:
            index = match.start()
            return desc[:index].strip(), desc[index:].strip()
        else:
            return desc, ''

    # Applying the function to split 'Generic Description'
    data_cleaned['Generic Name'], data_cleaned['Form'] = zip(*data_cleaned['Generic Description'].apply(split_description))
    data_cleaned.drop(columns=['Generic Description'], inplace=True)

    # Removing rows where 'Generic Name' is empty or whitespace
    data_cleaned = data_cleaned[data_cleaned['Generic Name'].str.strip() != '']

    # Function to split the description into name and size
    def split_description_on_number(desc):
        match = re.search(r'\d', desc)
        if match:
            index = match.start()
            return desc[:index].strip(), desc[index:].strip()
        else:
            return desc, ''

    # Applying the function to split 'Description'
    data_cleaned['Name'], data_cleaned['Size'] = zip(*data_cleaned['Description'].apply(split_description_on_number))
    data_cleaned.drop(columns=['Description'], inplace=True)

    data_cleaned
    return data_cleaned

In [2]:
def ExactDrugAlgoFunction(drug_code, data):
    # Item number of the drug to run the similarity test on
    reference_item_number = drug_code

    # Find and print the row for the given reference_item_number
    matching_row = data[data['Item Number – 8 digit'] == reference_item_number]
    

    # Find the Generic Name for the given reference_item_number
    reference_generic_name = data.loc[data['Item Number – 8 digit'] == reference_item_number, 'Generic Name'].iloc[0]


    # Create a copy of the dataframe filtered by Generic Name
    data_generic = data[data['Generic Name'] == reference_generic_name].copy()

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    # Selecting the reference item
    reference_item = data_generic[data_generic['Item Number – 8 digit'] == reference_item_number]
    if reference_item.empty:
        return "Reference item not found in the dataset."

    # Extracting the form of the reference item
    reference_form = reference_item.iloc[0]['Form']
    forms = data_generic['Form'].tolist()
    forms.insert(0, reference_form)

    # Vectorizing the forms using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(forms)

    # Calculating cosine similarity
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    data_generic['Similarity'] = cosine_similarities

    # Filtering the dataset to show only items with a similarity score above 0.9
    similarity_items = data_generic[data_generic['Similarity'] > 0].copy()

    # Extracting the size of the reference item
    reference_size = reference_item.iloc[0]['Size']
    sizes = similarity_items['Size'].tolist()
    sizes.insert(0, reference_size)

    # Vectorizing the sizes using TF-IDF
    tfidf_matrix_sizes = vectorizer.fit_transform(sizes)

    # Calculating cosine similarity for sizes
    cosine_similarities_sizes = cosine_similarity(tfidf_matrix_sizes[0:1], tfidf_matrix_sizes[1:]).flatten()
    similarity_items['Size Similarity'] = cosine_similarities_sizes

    # Remove input item
    similarity_items = similarity_items[similarity_items['Item Number – 8 digit'] != reference_item_number]

    # Define true similarity
    w1 = 1
    w2 = 1
    similarity_items['True Similarity'] = (w1 * similarity_items['Similarity'] + w2 * similarity_items['Size Similarity']) / (w1 + w2)

    # Cleaning and sort the data
    similarity_items = similarity_items.drop(columns=['Similarity', 'Size Similarity'])
    similarity_items = similarity_items.sort_values(by=['True Similarity'], ascending=False)
    
    
    return similarity_items, matching_row

In [3]:
import re

def addDosageInfo(inputData):
    def extractDosage(size_str):
        # Ensure the input is a string
        size_str = str(size_str)
        # Initialize a dictionary to hold the extracted values
        extracted_values = {'MG': None, '%': None, 'ML': None, 'GM': None, 'MCG': None, 'M': None, 'OZ': None, 'IU': None, 'MEQ': None, 'UN': None, 'MM': None, 'HR': None, 'MMOL': None, 'KG': None, 'BP': None, 'L': None, 'CM': None, 'CC': None, 'CAL': None, 'LB': None, 'IN': None, 'GR': None, 'GAL': None, 'LT': None, 'USP': None, 'G': None, 'MU': None}

        # Turn string to uppercase for case-insensitive matching
        size_str = size_str.upper()

        # Change 'X' to ' * ' for easier matching
        size_str = size_str.replace('X', ' * ')

        # Change '/' to ' / ' for easier matching
        size_str = size_str.replace('/', ' / ')
        
        # Simplified logic for MG, %, ML extraction
        # Looks for a number (with optional decimal) immediately before the unit, with optional space
        for unit in ['MG', '%', 'ML', 'GM', 'MCG', 'OZ', 'IU', 'MEQ', 'UN', 'MM', 'HR', 'MMOL', 'KG', 'BP', 'L', 'CM', 'CC', 'CAL', 'LB', 'IN', 'GR', 'GAL', 'LT', 'USP', 'MU']:
            pattern = r'(\d+(?:\.\d*)?)\s*{}'.format(unit)
            match = re.search(pattern, size_str, re.IGNORECASE)
            if match and 'X' not in match.group(1):
                # Convert matched value to float and assign to the correct unit
                extracted_values[unit] = float(match.group(1))

        # If no MG, ML, MCG, MEQ, MM, MMOL, MU, GM, GR, GAL are not found, check for M and G
        if extracted_values['MG'] is None and extracted_values['ML'] is None and extracted_values['MCG'] is None and extracted_values['MEQ'] is None and extracted_values['MM'] is None and extracted_values['MMOL'] is None and extracted_values['MU'] is None and extracted_values['GM'] is None and extracted_values['GR'] is None and extracted_values['GAL'] is None:
            # Check for M and G
            for unit in ['M', 'G']:
                pattern = r'(\d+(?:\.\d*)?)\s*{}'.format(unit)
                match = re.search(pattern, size_str, re.IGNORECASE)
                if match and 'X' not in match.group(1):
                    # Convert matched value to float and assign to the correct unit
                    extracted_values[unit] = float(match.group(1))
        
        return extracted_values

    # Apply the adjusted function to extract all values
    df_updated_extracted = inputData['Size'].apply(extractDosage)

    # Update the DataFrame with the new extracted values
    inputData['MG'] = df_updated_extracted.apply(lambda x: x['MG'])
    inputData['%'] = df_updated_extracted.apply(lambda x: x['%'])
    inputData['ML'] = df_updated_extracted.apply(lambda x: x['ML'])
    inputData['GM'] = df_updated_extracted.apply(lambda x: x['GM'])
    inputData['MCG'] = df_updated_extracted.apply(lambda x: x['MCG'])
    inputData['M'] = df_updated_extracted.apply(lambda x: x['M'])
    inputData['OZ'] = df_updated_extracted.apply(lambda x: x['OZ'])
    inputData['IU'] = df_updated_extracted.apply(lambda x: x['IU'])
    inputData['MEQ'] = df_updated_extracted.apply(lambda x: x['MEQ'])
    inputData['UN'] = df_updated_extracted.apply(lambda x: x['UN'])
    inputData['MM'] = df_updated_extracted.apply(lambda x: x['MM'])
    inputData['HR'] = df_updated_extracted.apply(lambda x: x['HR'])
    inputData['MMOL'] = df_updated_extracted.apply(lambda x: x['MMOL'])
    inputData['KG'] = df_updated_extracted.apply(lambda x: x['KG'])
    inputData['BP'] = df_updated_extracted.apply(lambda x: x['BP'])
    inputData['L'] = df_updated_extracted.apply(lambda x: x['L'])
    inputData['CM'] = df_updated_extracted.apply(lambda x: x['CM'])
    inputData['CC'] = df_updated_extracted.apply(lambda x: x['CC'])
    inputData['CAL'] = df_updated_extracted.apply(lambda x: x['CAL'])
    inputData['LB'] = df_updated_extracted.apply(lambda x: x['LB'])
    inputData['IN'] = df_updated_extracted.apply(lambda x: x['IN'])
    inputData['GR'] = df_updated_extracted.apply(lambda x: x['GR'])
    inputData['GAL'] = df_updated_extracted.apply(lambda x: x['GAL'])
    inputData['LT'] = df_updated_extracted.apply(lambda x: x['LT'])
    inputData['USP'] = df_updated_extracted.apply(lambda x: x['USP'])
    inputData['G'] = df_updated_extracted.apply(lambda x: x['G'])
    inputData['MU'] = df_updated_extracted.apply(lambda x: x['MU'])

    # Perform the conversions
    inputData['Total_MG'] = (inputData['MG'].fillna(0) +
                            inputData['GM'].fillna(0) * 1000 +
                            inputData['KG'].fillna(0) * 1000000 +
                            inputData['OZ'].fillna(0) * 28349.5 +
                            inputData['LB'].fillna(0) * 453592 +
                            inputData['GR'].fillna(0) * 1000 + 
                            inputData['G'].fillna(0) * 1000)
    inputData['Total_MG'] = inputData['Total_MG'].replace(0.0, np.nan)

    inputData['Total_ML'] = (inputData['ML'].fillna(0) +
                            inputData['L'].fillna(0) * 1000 +
                            inputData['GAL'].fillna(0) * 3785.41 +
                            inputData['LT'].fillna(0) * 1000 +
                            inputData['CC'].fillna(0) * 1)
    inputData['Total_ML'] = inputData['Total_ML'].replace(0.0, np.nan)

    inputData['Total_MM'] = (inputData['MM'].fillna(0) +
                            inputData['CM'].fillna(0) * 10 +
                            inputData['IN'].fillna(0) * 25.4)
    inputData['Total_MM'] = inputData['Total_MM'].replace(0.0, np.nan)

    # List of columns to drop (all the original measurement columns)
    cols_to_drop = ['MG', 'ML', 'GM', 'KG', 'OZ', 'LB', 'GR', 'L', 'GAL', 'LT', 'CC', 'MM', 'CM', 'IN', 'G']

    # Drop the original measurement columns
    inputData.drop(columns=cols_to_drop, inplace=True)

    # Now inputData contains only the totalized columns and any other non-related columns
    return inputData
    

In [5]:
import pandas as pd

def getReplacements(input, data, isMultiple = False):
    units = ['%', 'MCG', 'M', 'IU', 'MEQ', 'UN', 'HR', 'MMOL', 'BP', 'CAL', 'USP', 'MU', 'Total_ML', 'Total_MM', 'Total_MG']
    # Define a function to classify the basic form based on the units present
    def classify_basic_form(row):
        if pd.notna(row['Total_MG']) and pd.isna(row[units]).all():
            return 'Solid'
        # Add more classifications as needed based on other units
        else:
            return 'Other/Undefined'
        
    def isMultipleOf(num, multiple):
        return num % multiple == 0
    
    # Remove items with a True Similarity less that 0.5
    data = data[data['True Similarity'] > 0.5]

    # Apply the function to the dataset
    data = addDosageInfo(data)

    # Add dosage info to the input
    input = addDosageInfo(input)

    # Apply the classification function to each row
    data['Basic Form'] = data.apply(classify_basic_form, axis=1)

    # Apply the classification function to intput
    input['Basic Form'] = input.apply(classify_basic_form, axis=1)

    if input['Basic Form'].iloc[0] == 'Solid' and isMultiple:
        print('Solid')
        solids_data = data[data['Basic Form'] == 'Solid']

        # Remove items with a Total MG bigger than the input Total MG
        solids_data = solids_data[solids_data['Total_MG'] <= input['Total_MG'].iloc[0]]

        # Remove items where the input Total MG is not a multiple of the item's Total MG
        solids_data = solids_data[solids_data['Total_MG'].apply(lambda x: isMultipleOf(input['Total_MG'].iloc[0], x))]

        result = solids_data
    else:
        print('Not Solid or rule off')

        # Remove items where all units are not equal to the input units
        units = ['%', 'MCG', 'M', 'IU', 'MEQ', 'UN', 'HR', 'MMOL', 'BP', 'CAL', 'USP', 'MU', 'Total_ML', 'Total_MM', 'Total_MG']
        for unit in units:
            result = data[data[unit] == input[unit].iloc[0]]

    return result


In [9]:

filename = 'Daily Snapshot.csv'

# Clean the data
data_cleaned = cleanData(filename)

# Similarity
data, input = ExactDrugAlgoFunction(10000082, data_cleaned)

# get replacements
replacements = getReplacements(input, data, False)

replacements
    


Not Solid or rule off


/var/folders/rd/69bqgjk125d_y7cgrqxqwp4h0000gn/T/ipykernel_56102/2637827366.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputData['MG'] = df_updated_extracted.apply(lambda x: x['MG'])
/var/folders/rd/69bqgjk125d_y7cgrqxqwp4h0000gn/T/ipykernel_56102/2637827366.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputData['%'] = df_updated_extracted.apply(lambda x: x['%'])
/var/folders/rd/69bqgjk125d_y7cgrqxqwp4h0000gn/T/ipykernel_56102/2637827366.py:46: SettingWithCopyWarning: 
A value is trying to

,Item Number – 8 digit,NDC Number,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,...,HR,MMOL,BP,CAL,USP,MU,Total_MG,Total_ML,Total_MM,Basic Form
36541,10229260,43547043703,30,1,133305,9233,0,10000,N,entecavir,...,None,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined
28613,10186738,51991089633,30,1,133305,18466,0,20000,N,entecavir,...,None,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined
24759,10184880,68382092106,30,1,133305,7239,0,7840,N,entecavir,...,None,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined
19886,10249686,42806065930,30,1,133305,831,0,900,N,entecavir,...,None,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined
15636,10172844,69097042502,30,1,133305,5910,0,6400,N,entecavir,...,None,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined
12321,10161781,31722083430,30,1,133297,2720,0,4000,C,entecavir,...,None,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined
12120,10161050,65862084230,30,1,133305,5910,0,6400,N,entecavir,...,None,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined


## Goal

The current goal is to list the out items in terms of a weighted average of the 3 prices. The prices are taken from the CSV file and the weights will be given by the user. The weights describe the percentage amount of orders placed at each price.

In [10]:
#Set the weights for the average (as percentages)
w1 = 0.5
w2 = 0.25
w3 = 0.25

In [11]:
#Calculate the average price
replacements['Average Price'] = (w1 * replacements['AWP Price'] + w2 * replacements['Acquisition Price'] + w3 * replacements['WAC Price']) / (w1 + w2 + w3)

#Sort the replacements by the average price
replacements = replacements.sort_values(by='Average Price')

replacements

,Item Number – 8 digit,NDC Number,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,...,MMOL,BP,CAL,USP,MU,Total_MG,Total_ML,Total_MM,Basic Form,Average Price
19886,10249686,42806065930,30,1,133305,831,0,900,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,67085.25
12321,10161781,31722083430,30,1,133297,2720,0,4000,C,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,68328.50
15636,10172844,69097042502,30,1,133305,5910,0,6400,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,69730.00
12120,10161050,65862084230,30,1,133305,5910,0,6400,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,69730.00
24759,10184880,68382092106,30,1,133305,7239,0,7840,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,70422.25
36541,10229260,43547043703,30,1,133305,9233,0,10000,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,71460.75
28613,10186738,51991089633,30,1,133305,18466,0,20000,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,76269.00


In [12]:
# Save the replacements to a CSV file
replacements.to_csv('replacements.csv', index=False)

### Finalize as function

In [13]:
def addPriceAverage(w1, w2, w3, replacements):
    #Calculate the average price
    replacements['Average Price'] = (w1 * replacements['AWP Price'] + w2 * replacements['Acquisition Price'] + w3 * replacements['WAC Price']) / (w1 + w2 + w3)

    #Sort the replacements by the average price
    replacements = replacements.sort_values(by='Average Price')

    return replacements

In [14]:
# Test the function
addPriceAverage(0.5, 0.25, 0.25, replacements)

,Item Number – 8 digit,NDC Number,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,...,MMOL,BP,CAL,USP,MU,Total_MG,Total_ML,Total_MM,Basic Form,Average Price
19886,10249686,42806065930,30,1,133305,831,0,900,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,67085.25
12321,10161781,31722083430,30,1,133297,2720,0,4000,C,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,68328.50
15636,10172844,69097042502,30,1,133305,5910,0,6400,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,69730.00
12120,10161050,65862084230,30,1,133305,5910,0,6400,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,69730.00
24759,10184880,68382092106,30,1,133305,7239,0,7840,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,70422.25
36541,10229260,43547043703,30,1,133305,9233,0,10000,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,71460.75
28613,10186738,51991089633,30,1,133305,18466,0,20000,N,entecavir,...,None,None,None,None,None,1.0,NaN,NaN,Other/Undefined,76269.00
